In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Model parameters
# Arrival rates for each class (scaled down by factor 50)
lambda_rates = np.array([1, 2])  # [Class1, Class2] base rates
lambda_rates = lambda_rates / 50  # Scale down: [0.02, 0.04]

# Service time matrix D(i,r): mean service time of class r at station i
# Station 1: Class1=10, Class2=5
# Station 2: Class1=5,  Class2=9
D = np.array([[10, 5],
              [5, 9]])

# Delay station service times Z(r): mean service time of class r at delay
Z = np.array([91, 92])  # [Class1=91, Class2=92]

print("Model parameters:")
print(f"Arrival rates: {lambda_rates}")
print(f"Service time matrix D:\n{D}")
print(f"Delay service times Z: {Z}")

Model parameters:
Arrival rates: [0.02 0.04]
Service time matrix D:
[[10  5]
 [ 5  9]]
Delay service times Z: [91 92]


In [3]:
# Create tandem PS network with infinite servers
model = Network.tandemPsInf(lambda_rates, D, Z)

print(f"Tandem PS network created with {model.get_number_of_nodes()} nodes and {model.get_number_of_classes()} classes")
print(f"Node names: {[node.get_name() for node in model.nodes()]}")
print(f"Class names: {[cls.get_name() for cls in model.classes()]}")

Tandem PS network created with 5 nodes and 2 classes
Node names: ['Source', 'Delay1', 'Queue2', 'Queue3', 'Sink']
Class names: ['Class1', 'Class2']


In [4]:
# Aligned with JAR test scenarios for oqn_oneline# JAR tests: Fluid() only# Solve with Fluid solver (matches JAR)solver_fluid = FLD(model)avg_table_fluid = solver_fluid.avg_table()print(avg_table_fluid)

In [5]:
# Original JMT comparison commented out to align with JAR (Fluid only)# # solver_jmt = JMT(model)# avg_table_jmt = solver_jmt.avg_table()# print(avg_table_jmt)

In [6]:
# Performance analysis updated to use Fluid results only (JAR alignment)# Class-specific analysisfor class_name in ['Class1', 'Class2']:    class_metrics = avg_table_fluid[avg_table_fluid['JobClass'] == class_name]    print(f"\n{class_name}:")        total_resp_time = class_metrics['RespT'].sum()    total_utilization = class_metrics['Util'].sum()    avg_throughput = class_metrics['Tput'].iloc[0] if len(class_metrics) > 0 else 0        print(f"  Total response time: {total_resp_time:.4f}")    print(f"  Total utilization: {total_utilization:.4f}")    print(f"  Throughput: {avg_throughput:.6f}")        # Per-station breakdown    for _, row in class_metrics.iterrows():        if row['RespT'] > 0:  # Only show stations with non-zero response time            print(f"    {row['Station']}: RespT={row['RespT']:.4f}, Util={row['Util']:.4f}")# System totalstotal_system_tput = avg_table_fluid['Tput'].sum()print(f"\nTotal system throughput: {total_system_tput:.6f}")

In [7]:
# Little's Law verification updated to use Fluid results only (JAR alignment)for class_name in ['Class1', 'Class2']:    class_metrics = avg_table_fluid[avg_table_fluid['JobClass'] == class_name]    if len(class_metrics) > 0:        total_qlen = class_metrics['QLen'].sum()        total_resp_time = class_metrics['RespT'].sum()        throughput = class_metrics['Tput'].iloc[0]                # Little's Law: L = λ * W        expected_qlen = throughput * total_resp_time                print(f"\n{class_name}:")        print(f"  Observed queue length: {total_qlen:.6f}")        print(f"  Expected (λ*W): {expected_qlen:.6f}")        print(f"  Difference: {abs(total_qlen - expected_qlen):.8f}")